Fall 2021<br>
Modified from Theo Hartsook from NRES 782 <br>
Goal: create JSON pipelines to save las files <br>

See: http://www.asprs.org/wp-content/uploads/2019/07/LAS_1_4_r15.pdf page 19 for classification information


**ASPRS Standard Point Classes (Point Data Record Formats 6-10):**

0 Created, Never Classified See note4 <br>
1 Unclassified<br>
2 Ground<br>
3 Low Vegetation<br>
4 Medium Vegetation<br>
5 High Vegetation<br>
6 Building<br>
7 Low Point (Noise)<br>
8 Reserved<br>
9 Water<br>
10 Rail<br>
11 Road Surface<br>
12 Reserved<br>
13 Wire – Guard (Shield)<br>
14 Wire – Conductor (Phase)<br>
15 Transmission Tower<br>
16 Wire-Structure Connector e.g., insulators<br>
17 Bridge Deck<br>
18 High Noise<br>
19 Overhead Structure e.g., conveyors, mining equipment, traffic
lights<br>
20 Ignored Ground e.g., breakline proximity<br>
21 Snow<br>
22 Temporal Exclusion Features excluded due to changes over
time between data sources – e.g., water
levels, landslides, permafrost<br>
23-63 Reserved<br>
64-255 User Definable<br>

In [1]:
# import necessary files

import json # where we will save the json files to run a pipeline
import os 
import subprocess # allows us to run command line commands
import pdal
import rasterio
import pyproj
from osgeo import gdal
import glob
#os.chdir('/')
# file directory path on bigger computer:
# '\\\Harpold-lab1pc\\f\\lidar'

In [2]:
os.chdir('/Volumes/cpiske')
os.getcwd()

'/Volumes/cpiske'

In [3]:
# # test docker
# sudo docker run -v /Users/carapiske/Desktop/test_las:/input pdal/pdal touch /input/hey.txt

## Get info
Generally, we can either use our local pdal package or pull from a docker image. We'll practice both below but prefer the docker method

In [3]:
# define a test file (in this case: /mcc_part_b_tile_004_000.las)

#input_path = r'PDAL_testFiles_tutorials/test_las/SCB/NCALM_testFiles/ot_736000_4366000.laz' #'PDAL_testFiles_tutorials/test_las/SCB/NCALM_testFiles' # define input las path (not filename)
input_las = 'lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/reproject/20160518_toNAD83/toNAVD88/734736_4364742.las'
#nput_las = 'PDAL_testFiles_tutorials/test_las/SCB/NCALM_testFiles/ot_736000_4363000.laz'
#input_las_test = "PDAL_testFiles_tutorials/test_las/SCB/reproject/NCALM2014_toWGS84/ot_736000_4363000.laz"
#input_las = r'PDAL_testFiles_tutorials/test_las/SCB/NCALM_testFiles/ot_736000_4366000.laz'#'lidar_processing/PDAL_testFiles_tutorials/test_las/ncalm_2014_732000_4373000.las' # define input las full file path
#input_path_docker = input_path + ':/input'
#input_las = "lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/ASO_testFiles/WGS84/20160518/736236_4366272.las"
input_las_docker = '/input/'+'/mcc_part_b_tile_004_000.las'
# input_path = 'PDAL_testFiles_tutorials/test_las/SCB/NCALM_testFiles' #'PDAL_testFiles_tutorials/test_las/SCB/NCALM_testFiles' # define input las path (not filename)
# input_las = 'PDAL_testFiles_tutorials/test_las/SCB/NCALM_testFiles/ot_736000_4363000.laz'#'lidar_processing/PDAL_testFiles_tutorials/test_las/ncalm_2014_732000_4373000.las' # define input las full file path
# input_path_docker = input_path + ':/input'
# input_las_docker = '/input/'+'/ot_736000_4363000.laz'

windows

In [4]:
# # define a test file (in this case: /mcc_part_b_tile_004_000.las)
# input_path = 'lidar_processing\\PDAL_testFiles_tutorials\\test_las' # define input las path (not filename)
# input_las = 'lidar_processing\\PDAL_testFiles_tutorials\\test_las\\ASO\\ICB\\USCAMB20140324f1a1_-_Scanner_2_-_140324_203340_2_-_originalpoints_dem_filter.las'#'lidar_processing/PDAL_testFiles_tutorials/test_las/ncalm_2014_732000_4373000.las' # define input las full file path

# input_path_docker = input_path + ':\\input'
# input_las_docker = '\\input\\'+'USCAMB20140324f1a1_-_Scanner_2_-_140324_203340_2_-_originalpoints_dem_filter.las'

In [5]:
pdal_info_command = ['pdal', 'info', input_las]
#pdal_info_command_Docker = ['docker', 'run', '-v', input_path_docker, 'pdal/pdal', 'pdal', 'info', input_las_docker]
pdal_metadata_command = ['pdal', 'info', input_las, '--metadata']

In [36]:
# # terminal command using docker
# subprocess.run(['docker', 'run', '-v', '/Users/carapiske/Desktop/test_las:/input', 'pdal/pdal', 'pdal', 'info', '/input/mcc_part_b_tile_004_000.las'])
# # or 
# subprocess.run(pdal_info_command_Docker)
# # or use local pdal
# subprocess.run(pdal_info_command)

In [49]:
pdal_info_results = subprocess.run(pdal_metadata_command, stdout = subprocess.PIPE) # stout (standard out), PIPE indicates that a new pipe to the child should be created
pdal_info_dict = json.loads(pdal_info_results.stdout.decode())

In [50]:
pdal_info_dict['metadata']['minx'] - pdal_info_dict['metadata']['maxx']

-1499.6199999999953

In [6]:
subprocess.run(pdal_metadata_command)

{
  "file_size": 814502,
  "filename": "lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/reproject/20160518_toNAD83/toNAVD88/734736_4364742.las",
  "metadata":
  {
    "comp_spatialreference": "COMPD_CS[\"unknown\",PROJCS[\"NAD83 / UTM zone 10N\",GEOGCS[\"NAD83\",DATUM[\"North_American_Datum_1983\",SPHEROID[\"GRS 1980\",6378137,298.257222101,AUTHORITY[\"EPSG\",\"7019\"]],AUTHORITY[\"EPSG\",\"6269\"]],PRIMEM[\"Greenwich\",0,AUTHORITY[\"EPSG\",\"8901\"]],UNIT[\"degree\",0.0174532925199433,AUTHORITY[\"EPSG\",\"9122\"]],AUTHORITY[\"EPSG\",\"4269\"]],PROJECTION[\"Transverse_Mercator\"],PARAMETER[\"latitude_of_origin\",0],PARAMETER[\"central_meridian\",-123],PARAMETER[\"scale_factor\",0.9996],PARAMETER[\"false_easting\",500000],PARAMETER[\"false_northing\",0],UNIT[\"metre\",1,AUTHORITY[\"EPSG\",\"9001\"]],AXIS[\"Easting\",EAST],AXIS[\"Northing\",NORTH],AUTHORITY[\"EPSG\",\"26910\"]],VERT_CS[\"unknown\",VERT_DATUM[\"unknown\",2005],UNIT[\"metre\",1,AUTHORITY[\"EPSG\",\"9001\"]],AXIS[\"U

CompletedProcess(args=['pdal', 'info', 'lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/reproject/20160518_toNAD83/toNAVD88/734736_4364742.las', '--metadata'], returncode=0)

## Rename

In [66]:
# pathname = 'PDAL_testFiles_tutorials/test_las/SCB/ASO_testFiles/WGS84/20160518/'
# for filepath in glob.iglob(r'PDAL_testFiles_tutorials/test_las/SCB/ASO_testFiles/WGS84/20160518/*.las'):
#     pdal_info_command = ['pdal', 'info', filepath, '--metadata']
#     pdal_info_results = subprocess.run(pdal_info_command, stdout = subprocess.PIPE) # stout (standard out), PIPE indicates that a new pipe to the child should be created
#     pdal_info_dict = json.loads(pdal_info_results.stdout.decode())
#     name = pathname+str(round(pdal_info_dict['metadata']['minx'])) +"_"+ str(round(pdal_info_dict['metadata']['miny']))+".las"
#     os.rename(filepath, name)

In [67]:
# name = str(round(pdal_info_dict['metadata']['minx'])) +"_"+ str(round(pdal_info_dict['metadata']['miny']))


## Datum Conversions

In [25]:
test_datum_conversion = 'lidar_processing/python_scripts/PDAL_workflow/JSON/test_datum_conversion.json'

From PDAL library: https://pdal.io/tutorial/grid-shift.html

In [76]:
# {
#     "pipeline": 
#         [
#             {
#                  "type": "readers.las",
#                  "filename": "#"
#             },
#             {
#                  "type": "filters.reprojection",
#                  "in_srs": "EPSG:2193",
#                  "out_srs": "EPSG:2193"
#             },
#             {
#                  "type": "writers.las",
#                  "filename": "#",
#                  "a_srs": "EPSG:2193", 
#                  "forward": "all"
#             }
#         ]
# }

In [77]:
input_las
output_las = "lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/reproject/20160518_toNAD83/736236_4366272.las"

In [96]:
input_las

'lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/ASO_testFiles/WGS84/20160518/736236_4366272.las'

In [79]:
# reader_dict = {"type":"readers.las",
#                "filename":input_las,
#               "spatialreference":"EPSG:26910"}

# reproject_dict = {"type":"filters.reprojection",
#         "in_srs":"EPSG:26910",
#         "out_srs":"EPSG:4326"}
# writer_dict = {"type":"writers.las",
#         "filename":output_las}

# pipeline_list = [reader_dict, reproject_dict, writer_dict]
# pipeline_dict = {'pipeline' : pipeline_list}
# with open(test_datum_conversion, 'w') as out:
#     json.dump(pipeline_dict, out, indent=4)



In [24]:
reader_dict = {"type":"readers.las"}

reproject_dict = {"type":"filters.reprojection","out_srs":"EPSG:26910"}
writer_dict = {"type":"writers.las",
               "a_srs":"EPSG:26910",
               "scale_x":0.00000001,
               "scale_y":0.00000001,
              "offset_x":"auto",
              "offset_y":"auto"}

pipeline_list = [reader_dict, reproject_dict, writer_dict]
pipeline_dict = {'pipeline' : pipeline_list}
with open(test_datum_conversion, 'w') as out:
    json.dump(pipeline_dict, out, indent=4)


In [98]:
pdal_commands = ['pdal', 'translate', input_las, output_las, '--json', test_datum_conversion]
subprocess.run(pdal_commands)

CompletedProcess(args=['pdal', 'translate', 'lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/ASO_testFiles/WGS84/20160518/736236_4366272.las', 'lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/reproject/20160518_toNAD83/736236_4366272.las', '--json', 'lidar_processing/python_scripts/PDAL_workflow/JSON/test_datum_conversion.json'], returncode=0)

In [26]:
pathname = 'lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/ASO_testFiles/WGS84/20160518/'
output_pathname = "lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/reproject/20160518_toNAD83/"
onlyfiles = [f for f in os.listdir(pathname) if os.path.isfile(os.path.join(pathname, f))]
for file in onlyfiles:
    input_las = pathname + file
    output_las = output_pathname + file
    pdal_commands = ['pdal', 'translate', input_las, output_las, '--json', test_datum_conversion]
    subprocess.run(pdal_commands)

In [107]:
input_las_reproj = "lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/reproject/20160518_toNAD83/736236_4366272.las"
output_las_vreproj = "lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/reproject/20160518_toNAD83/736236_4366272_v.las"

Vertical

In [28]:
vertical_datum_json = "lidar_processing/python_scripts/PDAL_workflow/JSON/vertical_datum_conversion.json"
reader_dict = {"type":"readers.las"}

reproject_dict = {"type":"filters.reprojection",
                  "in_srs":"EPSG:26910",
                  "out_srs":"+init=epsg:26910 +geoidgrids=/Volumes/cpiske/lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/reproject/g2012bu5.gtx"}

writer_dict = {"type":"writers.las",
               "a_srs":"EPSG:5498"} #EPSG:26910+5703

pipeline_list = [reader_dict, reproject_dict, writer_dict]
pipeline_dict = {'pipeline' : pipeline_list}
with open(vertical_datum_json, 'w') as out:
    json.dump(pipeline_dict, out, indent=4)


In [116]:
pdal_commands = ['pdal', 'translate', input_las_reproj, output_las_vreproj, '--json', vertical_datum_json]
subprocess.run(pdal_commands)

Warning 1: +init=epsg:XXXX syntax is deprecated. It might return a CRS with a non-EPSG compliant axis order.


CompletedProcess(args=['pdal', 'translate', 'lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/reproject/20160518_toNAD83/736236_4366272.las', 'lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/reproject/20160518_toNAD83/736236_4366272_v.las', '--json', 'lidar_processing/python_scripts/PDAL_workflow/JSON/vertical_datum_conversion.json'], returncode=0)

In [11]:
pathname = 'lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/ASO_testFiles/WGS84/20160518/'
onlyfiles = [f for f in os.listdir(pathname) if os.path.isfile(os.path.join(pathname, f))]

In [30]:
pathname = 'lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/reproject/20160518_toNAD83/'
output_pathname = "lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/reproject/20160518_toNAD83/toNAVD88/"
onlyfiles = [f for f in os.listdir(pathname) if os.path.isfile(os.path.join(pathname, f))]
for file in onlyfiles:
    input_las = pathname + file
    output_las = output_pathname + file
    pdal_commands = ['pdal', 'translate', input_las, output_las, '--json', vertical_datum_json]
    subprocess.run(pdal_commands)

Warning 1: +init=epsg:XXXX syntax is deprecated. It might return a CRS with a non-EPSG compliant axis order.
Warning 1: +init=epsg:XXXX syntax is deprecated. It might return a CRS with a non-EPSG compliant axis order.
Warning 1: +init=epsg:XXXX syntax is deprecated. It might return a CRS with a non-EPSG compliant axis order.
Warning 1: +init=epsg:XXXX syntax is deprecated. It might return a CRS with a non-EPSG compliant axis order.
Warning 1: +init=epsg:XXXX syntax is deprecated. It might return a CRS with a non-EPSG compliant axis order.
Warning 1: +init=epsg:XXXX syntax is deprecated. It might return a CRS with a non-EPSG compliant axis order.
Warning 1: +init=epsg:XXXX syntax is deprecated. It might return a CRS with a non-EPSG compliant axis order.
Warning 1: +init=epsg:XXXX syntax is deprecated. It might return a CRS with a non-EPSG compliant axis order.
Warning 1: +init=epsg:XXXX syntax is deprecated. It might return a CRS with a non-EPSG compliant axis order.
Warning 1: +init=ep

## Convert .las to .txt
see: https://pdal.io/stages/writers.text.html

In [177]:
# set up json file commands
output_txt = '/Volumes/cpiske/lidar_processing/python_scripts/PDAL/test_las/mcc_part_b_tile_004_000Test.asc'
output_json = 'lidar_processing/python_scripts/PDAL_workflow/JSON/las_to_txt.json'

# create a pipeline and save to a json file 

filter_dict = {'type':'readers.las',
               'override_srs': "EPSG:4326",
              'filename': input_las} # we are reading in a las file
rasterize_dict = {'type':'writers.las',
'format':'geojson',
'order':'X,Y,Z',
'keep_unspecified':'false',
'filename':output_txt}


pipeline_list = [filter_dict, rasterize_dict]
pipeline_dict = {'pipeline' : pipeline_list}
with open(output_json, 'w') as out:
    json.dump(pipeline_dict, out, indent=4)


In [72]:
json_path = 'lidar_processing/python_scripts/PDAL/JSON/las_to_txt.json'
pdal_commands = ['pdal', 'pipeline', json_path]
subprocess.run(pdal_commands)

proj_create_from_database: crs not found
proj_create_from_database: crs not found
proj_uom_get_info_from_database: unit of measure not found
proj_create_from_database: crs not found


CompletedProcess(args=['pdal', 'pipeline', 'lidar_processing/python_scripts/PDAL/JSON/las_to_txt.json'], returncode=0)

### .laz to .las

In [31]:
# # set up json file commands
# input_laz = 'lidar_processing/python_scripts/PDAL/test_las/ASO_USCAMB20180425f1a1_180425_1_dem_filter.laz'
# output_las = 'lidar_processing/python_scripts/PDAL/test_las/ASO_USCAMB20180425f1a1_180425_1_dem_filter.las'
# output_json = 'lidar_processing/python_scripts/PDAL/JSON/laz_to_las.json'

# # create a pipeline and save to a json file 

# filter_dict = {'type':'readers.las',
#                'filename': input_las} # we are reading in a las file
# translate_dict = {'type':'writers.las',
#                   "a_srs": "EPSG:4326",
#                   'filename':output_las}


# pipeline_list = [filter_dict, translate_dict]
# pipeline_dict = {'pipeline' : pipeline_list}
# with open(output_json, 'w') as out:
#     json.dump(pipeline_dict, out, indent=4)


In [32]:
# json_path = 'lidar_processing/python_scripts/PDAL/JSON/laz_to_las.json'
# pdal_commands = ['pdal', 'pipeline', json_path]
# subprocess.run(pdal_commands)

## Ground Filter Tutorial
Modified from https://pdal.io/tutorial/ground-filters.html <br>
Bradley Chambers

In [42]:
# name JSON file
output_json = 'lidar_processing/python_scripts/PDAL_workflow/JSON/ground_filter_tutorial.json'

# create a pipeline and save to a json file 
#last_val = True
filter_dict_reproj = {'type':'filters.reprojection', #PDAL’s default parameters are specified in meters, and individual filter stages typically assume that units are at least uniform in X, Y, and Z. Because data will not always be provided in this way, PDAL pipelines should account for any data reprojections and parameter scaling that are required from one dataset to the next
                      'in_srs':'EPSG:26910',
                      'out_srs': "EPSG:26910",}

filter_dict_assign = {'type':'filters.assign', # set the value of a dimension for all points to a provided value that pass a range filter
                      "assignment":"Classification[:]=0"} #  single option has been provided that specifies the dimension, range, and value to assign. In this case, we are stating that we would like to apply a value of 0 to the Classification dimension for every point

filter_dict_elm = {'type':'filters.elm'} # identify low noise points that can adversely affect ground segmentation algorithms, automatically assigns value of 7 

filter_dict_outlier = {'type':'filters.outlier'} #  two methods of outlier detection at the moment: radius and statistical. Both aim to identify points that are isolated and likely arise from noise sources, classify values as 7
# classify ground points
filter_dict_smrf = {"type":"filters.smrf",
                    "ignore":"Classification[7:7]"}#, # ignore outliers
                    #"slope":0.3,
                    #"window":16,
                    #"threshold":0.15,
                    #"scalar":1.2}

filter_dict_range = {"type":"filters.range",
                    "limits":"Classification[2:2]"}
#output_las = "lidar_processing/python_scripts/PDAL/test_las/ncalm_2014_732000_4373000_GFtutorial.las"


pipeline_list = [filter_dict_reproj,filter_dict_assign,filter_dict_elm, filter_dict_outlier, filter_dict_smrf, filter_dict_range]
pipeline_dict = {'pipeline' : pipeline_list}
with open(output_json, 'w') as out:
    json.dump(pipeline_dict, out, indent=4)

In [43]:
pdal_commands = ['pdal', 'translate', 'lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/NCALM_testFiles/ot_736000_4363000.laz', "lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/ground_filtered/NCALM/test/ot_736000_4363000.laz", '--json', output_json]
subprocess.run(pdal_commands)

CompletedProcess(args=['pdal', 'translate', 'lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/NCALM_testFiles/ot_736000_4363000.laz', 'lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/ground_filtered/NCALM/test/ot_736000_4363000.laz', '--json', 'lidar_processing/python_scripts/PDAL_workflow/JSON/ground_filter_tutorial.json'], returncode=0)

In [33]:
pathname = 'lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/NCALM_testFiles/'
output_pathname = "lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/ground_filtered/NCALM/simple/"
onlyfiles = [f for f in os.listdir(pathname) if os.path.isfile(os.path.join(pathname, f))]
for file in onlyfiles:
    input_las = pathname + file
    output_las = output_pathname + file
    pdal_commands = ['pdal', 'translate', input_las, output_las, '--json', output_json]
    subprocess.run(pdal_commands)

In [249]:
#gf_tutorial_command = ['pdal', 'translate', input_las, output_las, '--json', 'lidar_processing/python_scripts/PDAL_workflow/JSON/ground_filter_tutorial.json']

In [251]:
#subprocess.run(gf_tutorial_command)

In [35]:
pathname = 'lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/reproject/20160518_toNAD83/toNAVD88/'
output_pathname = "lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/ground_filtered/ASO_20160518/reproj_NAD83_NAVD88/"
onlyfiles = [f for f in os.listdir(pathname) if os.path.isfile(os.path.join(pathname, f))]
for file in onlyfiles:
    input_las = pathname + file
    output_las = output_pathname + file
    pdal_commands = ['pdal', 'translate', input_las, output_las, '--json', output_json]
    subprocess.run(pdal_commands)

In [34]:
# for i in [0, 1, 2, 3, 4, 5]:
#     input_las = 'lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/ASO_testFiles/NAD83_NAVD88/20160518/mcc_part_b_tile_004_00'+str(i) + '.las'
#     output_las = 'lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/ground_filtered/ASO_20160518/mcc_part_b_tile_004_00'+str(i)+'.las'
#     gf_tutorial_command = ['pdal', 'translate', input_las, output_las, '--json', 'lidar_processing/python_scripts/PDAL_workflow/JSON/ground_filter_tutorial.json']
#     subprocess.run(gf_tutorial_command)

## Rasterize

In [28]:
# name JSON file
rasterize_json = 'lidar_processing/python_scripts/PDAL_workflow/JSON/las_to_tif.json'
# input_las = 'lidar_processing/python_scripts/PDAL/test_las/mcc_part_b_tile_004_000.las' # define input las full file path
# output_las = 'lidar_processing/python_scripts/PDAL/test_las/mcc_part_b_tile_004_000_GFtutorial.las' # define input las full file path

# create a pipeline and save to a json file 
reader_dict = {'type':'readers.las'}
filter_gdal= {"type": "writers.gdal",
              'output_type': 'mean',
              'resolution': '1.0'}
#output_dtm = "lidar_processing/python_scripts/PDAL/test_file/ncalm_2014_732000_4373000_DTMtutorial.tif"


pipeline_list = [reader_dict,filter_gdal]
pipeline_dict = {'pipeline' : pipeline_list}
with open(rasterize_json, 'w') as out:
    json.dump(pipeline_dict, out, indent=4)

In [36]:
# Worked!
pathname = "lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/ground_filtered/NCALM/"
output_pathname = "lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/rasterize/NCALM/"
onlyfiles = [f for f in os.listdir(pathname) if os.path.isfile(os.path.join(pathname, f))]
for file in onlyfiles:
    writer = '--writers.gdal.filename='+output_pathname+file[:-4]+".tif"
    reader = '--readers.las.filename='+pathname+file
    rasterize_command = ['pdal', 'pipeline', rasterize_json, writer, reader]
    subprocess.run(rasterize_command)

test script

In [25]:
# # name JSON file
# rasterize_json = 'lidar_processing/python_scripts/PDAL_workflow/JSON/las_to_tif.json'
# # input_las = 'lidar_processing/python_scripts/PDAL/test_las/mcc_part_b_tile_004_000.las' # define input las full file path
# # output_las = 'lidar_processing/python_scripts/PDAL/test_las/mcc_part_b_tile_004_000_GFtutorial.las' # define input las full file path

# # create a pipeline and save to a json file 
# reader_dict = {'type':'readers.las'}
# filter_gdal= {"type": "writers.gdal",
#               'gdaldriver':'GTiff',
#               'output_type': 'mean',
#               'resolution': 1.0}
# #output_dtm = "lidar_processing/python_scripts/PDAL/test_file/ncalm_2014_732000_4373000_DTMtutorial.tif"


# pipeline_list = [reader_dict,filter_gdal]
# pipeline_dict = {'pipeline' : pipeline_list}
# with open(rasterize_json, 'w') as out:
#     json.dump(pipeline_dict, out, indent=4)

In [31]:
# See below for better solution

# pathname = "lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/ground_filtered/NCALM/"
# output_pathname = "lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/rasterize/NCALM/"
# onlyfiles = [f for f in os.listdir(pathname) if os.path.isfile(os.path.join(pathname, f))]
# for file in onlyfiles:
#     input_las = pathname + file
#     output_las = output_pathname + file[:-4]+".tif"
#     pdal_commands = ['pdal', 'translate', input_las, output_las, '--writers.gdal.resolution=1','--writers.gdal.output_type=mean']
#     subprocess.run(pdal_commands)

In [37]:
# didn't work because the pipeline wants an input/output

# pathname = "lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/ground_filtered/NCALM/"
# output_pathname = "lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/rasterize/NCALM/"
# onlyfiles = [f for f in os.listdir(pathname) if os.path.isfile(os.path.join(pathname, f))]
# for file in onlyfiles:
#     input_las = pathname + file
#     output_las = output_pathname + file[:-4]+".tif"
#     pdal_commands = ['pdal', 'translate', input_las, output_las, '--json', rasterize_json]
#     subprocess.run(pdal_commands)

In [276]:
# test new formatting
# input_las = 'lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/ground_filtered/ASO_20160417/mcc_part_b_tile_004_000.las'
# output_las = 'lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/rasterize/ASO_20160417/mcc_part_b_tile_004_000.tif'
# writer = '--writers.gdal.filename=lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/rasterize/ASO_20160417/mcc_part_b_tile_004_000.tif'
# reader = '--readers.las.filename=lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/ground_filtered/ASO_20160417/mcc_part_b_tile_004_000.las'
# rasterize_command = ['pdal', 'pipeline', 'lidar_processing/python_scripts/PDAL_workflow/JSON/las_to_tif.json', writer, reader]

In [281]:
# test on ASO data
# for i in [0, 1, 2, 3, 4, 5]:
#     writer = '--writers.gdal.filename=lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/rasterize/ASO_20160518/mcc_part_b_tile_004_00'+str(i)+'.tif'
#     reader = '--readers.las.filename=lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/ground_filtered/ASO_20160518/mcc_part_b_tile_004_00'+str(i)+'.las'
#     rasterize_command = ['pdal', 'pipeline', 'lidar_processing/python_scripts/PDAL_workflow/JSON/las_to_tif.json', writer, reader]
#     subprocess.run(rasterize_command)

# Raster Caluclations

In [287]:
for i in [0,1,2,3,4,5]:
    apr_elev = 'lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/rasterize/ASO_20160417/mcc_part_b_tile_004_00'+str(i)+'.tif'
    may_elev = 'lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/rasterize/ASO_20160518/mcc_part_b_tile_004_00'+str(i)+'.tif'
    output = 'lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/raster_subtract/mcc_part_b_tile_004_00'+str(i)+'.tif'
    raster_sub = ['gdal_calc.py', '-a', apr_elev, '-b', may_elev, '--calc="a - b"', '--outfile', output]
    subprocess.run(raster_sub)

0.. 3.. 7.. 11.. 15.. 19.. 23.. 26.. 30.. 34.. 38.. 42.. 46.. 50.. 53.. 57.. 61.. 65.. 69.. 73.. 76.. 80.. 84.. 88.. 92.. 96.. 100 - Done
0.. 3.. 7.. 11.. 15.. 19.. 23.. 26.. 30.. 34.. 38.. 42.. 46.. 50.. 53.. 57.. 61.. 65.. 69.. 73.. 76.. 80.. 84.. 88.. 92.. 96.. 100 - Done
0.. 3.. 7.. 11.. 15.. 19.. 23.. 26.. 30.. 34.. 38.. 42.. 46.. 50.. 53.. 57.. 61.. 65.. 69.. 73.. 76.. 80.. 84.. 88.. 92.. 96.. 100 - Done
0.. 3.. 7.. 11.. 15.. 19.. 23.. 26.. 30.. 34.. 38.. 42.. 46.. 50.. 53.. 57.. 61.. 65.. 69.. 73.. 76.. 80.. 84.. 88.. 92.. 96.. 100 - Done
0.. 3.. 7.. 11.. 15.. 19.. 23.. 26.. 30.. 34.. 38.. 42.. 46.. 50.. 53.. 57.. 61.. 65.. 69.. 73.. 76.. 80.. 84.. 88.. 92.. 96.. 100 - Done
Error! Dimensions of file lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/rasterize/ASO_20160518/mcc_part_b_tile_004_005.tif (145, 4) are different from other files (151, 4).  Cannot proceed


# Retiling

In [116]:
#docker run -v %cd%/input:/input -v %cd%/output:/output a8 pdal tile -i /input/*.las -o /output/ltbmu_tile#.las --length 500

In [115]:
pdal_command = ['pdal', 'tile', "PDAL_testFiles_tutorials/test_las/SCB/ASO_testFiles/WGS84/20160326/*.laz", 'PDAL_testFiles_tutorials/test_las/SCB/ASO_testFiles/WGS84/20160326/retile/retile_#.laz','length = 1000','buffer = 50']
subprocess.run(pdal_command)

CompletedProcess(args=['pdal', 'tile', 'PDAL_testFiles_tutorials/test_las/SCB/ASO_testFiles/WGS84/20160326/*.laz', 'PDAL_testFiles_tutorials/test_las/SCB/ASO_testFiles/WGS84/20160326/retile/retile_#.laz', 'length = 1000', 'buffer = 50'], returncode=0)

# Theo Code

In [17]:
input_las = 'SCB/kost_lidar_data/ASO_2016/2016_05_18/WGS84_G1762_to_NAD83_NAVD88/mcc_part_b_tile_004_000.las'

z_min = 0.15
z_max = 2
z_range = 'Z[' + str(z_min) + ':' + str(z_max) + ']'
output_raster = 'lidar_processing/python_scripts/PDAL/test_file/mcc_part_b_tile_004_000.tif'
output_json = 'lidar_processing/python_scripts/PDAL/JSON/las_to_tif.json'
resolution = 0.01

filter_dict = {'type':'filters.range', 'limits':z_range}
rasterize_dict = {'filename':output_raster,
'gdaldriver':'GTiff',
'output_type':'count',
'resolution':resolution,
'type': 'writers.gdal'}


pipeline_list = [input_las, filter_dict, rasterize_dict]
pipeline_dict = {'pipeline' : pipeline_list}
with open(output_json, 'w') as out:
    json.dump(pipeline_dict, out, indent=4)


In [16]:
input_las = 'SCB/kost_lidar_data/ASO_2016/2016_05_18/WGS84_G1762_to_NAD83_NAVD88/mcc_part_b_tile_004_000.las'

# z_min = 0.15
# z_max = 2
# z_range = 'Z[' + str(z_min) + ':' + str(z_max) + ']'
output_raster = 'lidar_processing/python_scripts/PDAL/test_file/mcc_part_b_tile_004_000.asc'
output_json = 'lidar_processing/python_scripts/PDAL/JSON/las_to_asc.json'
resolution = 0.01

filter_dict = {'type':'filters.range', 'limits':z_range}
rasterize_dict = {'filename':output_raster,
'gdaldriver':'XYZ',
'output_type':'count',
'resolution':resolution,
'type': 'writers.gdal'}


pipeline_list = [input_las, filter_dict, rasterize_dict]
pipeline_dict = {'pipeline' : pipeline_list}
with open(output_json, 'w') as out:
    json.dump(pipeline_dict, out, indent=4)


sudo docker run -v path_to_laz_folder:/input 0b pdal info /input/test.laz

where path_to_laz_folder is the path to the LAS/LAZ file (you just need the folder path, not the file path).

:/input is the new folder that will be created in your Docker container that will hold your point cloud.

0b is just the image id of pdal

/input/test.laz is the path to the point cloud in the Docker container.


In [ ]:
path_to_laz_folder = 'lidar_processing/python_scripts/PDAL/test_las'


In [ ]:
sudo docker run -v path_to_laz_folder:/input 0b pdal info /input/test.laz


In [ ]:
import json
import subprocess
import pdal

def assemblePipeline(input_las, list_of_dicts):
    pipeline_list = [input_las]
    pipeline_list.extend(list_of_dicts)
    pipeline_dict = {'pipeline' : pipeline_list}
    return pipeline_dict

def makeHeightFilter(height, buffer):
    z_min = height - buffer/2
    z_max = height + buffer/2
    z_range = 'Z[' + str(z_min) + ':' + str(z_max) + ']'
    heightDict = {'type':'filters.range', 'limits':z_range}
    return heightDict

def makeRasterizeFilter(output_raster, resolution, epsg):
    rasterize_dict = {'filename':output_raster,
                      'gdaldriver':'GTiff',
                      'output_type':'count',
                      'resolution':resolution,
                      'override_srs' : epsg,
                      'type': 'writers.gdal'}
    return rasterize_dict

def convertTifForPIL(input_raster, output_raster, epsg):
    ''' GDAL bindings are an alien concept to me, so I gave up and used
        subprocess.'''
    commands = ['gdal_translate', input_raster, output_raster, '-ot', 'Byte', '-a_srs', epsg]
    subprocess.run(commands)


def buildHeightSlice(input_las, height, buffer, output_raster, resolution, epsg, json_path=None):
    filter_dict = makeHeightFilter(height, buffer)
    rasterize_dict = makeRasterizeFilter(output_raster, resolution, epsg)
    filter_list = [filter_dict, rasterize_dict]
    pipeline_dict = assemblePipeline(input_las, filter_list)

    if json_path is not None:
        with open(json_path, 'w') as out:
            json.dump(pipeline_dict, out, indent=4)
        pdal_commands = ['pdal', 'pipeline', json_path]
        subprocess.run(pdal_commands)
    else:
        pdal_commands = json.dumps(pipeline_dict)
        pipeline = pdal.Pipeline(pdal_commands)
        pipeline.execute()

input_las = '/Users/theo/data/las/TLS_0244_20180612_01_v003_30m_clip_height_norm.las'
height = 1.37
buffer = 0.05
z_min = height - buffer/2
z_max = height + buffer/2
z_range = 'Z[' + str(z_min) + ':' + str(z_max) + ']'
temp_raster = '/Users/theo/Pictures/almost_cool.tif'
final_raster = '/Users/theo/Pictures/cool.tif'
resolution = 0.01
epsg = 'EPSG:3310'

buildHeightSlice(input_las, height, buffer, temp_raster, resolution, epsg)
convertTifForPIL(temp_raster, final_raster, epsg)

In [ ]:
import json
import subprocess
import pdal
import argparse

# Create flags for the user to utilize.
parser = argparse.ArgumentParser(description="Generate JSON pipeline to generate DTM from a point cloud.")
      
required = parser.add_argument_group('Required arguments')
required.add_argument('-crs', '--coordinate_system', required=True, action='store', help="EPSG code.")
required.add_argument('-i', '--infile', required=True, action='store', help="Input path to point cloud")
required.add_argument('-o', '--outfile', required=True, action='store', help="Output path.")
args = parser.parse_args()

def generateJSON(infile, list_of_dicts):
    pipeline_list = [infile]
    pipeline_list.extend(list_of_dicts)
    pipeline_dict = {'pipeline': pipeline_list}
    with open("pipeline.json", 'w') as out:
        json.dump(pipeline_dict, out, indent=4)

def generateDTM(epsg, infile, outfile):
    reproject_dict = {"type": "filters.reprojection",
                      "out_srs": "EPSG:{}".format(epsg)}
    reclassify_zero_dict = {"type": "filters.assign",
                       "assignment": "Classification[:]=0"}
    elm_dict = {"type": "filters.elm"}
    outlier_dict = {"type": "filters.outlier"}
    smrf_dict = {"type": "filters.smrf", "ignore": "Classification[7:7]",
                 "slope": 0.2, "window": 16, "threshold": 0.45, "scalar": 1.2}
    range_dict = {"type":"filters.range", "limits":"Classification[2:2]"}
    output_dict = {"filename": outfile, "gdaldriver": "GTiff", "output_type": "all", "resolution": 0.01, "type": "writers.gdal"}
    list_of_dicts = list([reproject_dict, reclassify_zero_dict, elm_dict, outlier_dict, smrf_dict, range_dict, output_dict])
    generateJSON(infile, list_of_dicts)
    pdal_cmds = ['pdal', 'pipeline', 'pipeline.json']
    subprocess.run(pdal_cmds)
    
generateDTM(args.coordinate_system, args.infile, args.outfile)